# 実験を開始する

これは、実験パッケージを用意して実験を行うためのノートブックです。  

## 1. 実験用の空リポジトリを作成する
実験リポジトリを作成する準備として、[リポジトリの作成方法](https://gin.g-node.org/G-Node/Info/wiki/Web+Interface#creating-a-repository)を参考にデータガバナンス機能で実験用の空リポジトリを作成してください。

1. [データガバナンス機能に遷移する](http://dg02.dg.rcos.nii.ac.jp/)

## 2. ステップ1で作成した空リポジトリに実験パッケージをコピーする

1. 以下のセルを実行すると入力フォームが現れます。ステップ1で作成した空リポジトリの名前を入力してください。

In [ ]:
repository_title = input()

In [ ]:
%cd ~/
tmp = !git remote -v
tmp = tmp[0]
tmp = tmp[tmp.find("http"):]
tmp = tmp.split("/")
tmp = tmp[0] + "//" + tmp[2] + "/" + tmp[3]

ssh_url = tmp + "/" + repository_title + ".git"
print(ssh_url)

2. 空リポジトリ名のフォルダを作成し、実験パッケージをコピーします。

In [ ]:
%%sh
echo "
machine dg02.dg.rcos.nii.ac.jp
login 【ユーザ名】
password 【パスワード】
" > ~/.netrc

In [ ]:
!mkdir ~/.tmp/
!git init ~/.tmp/$repository_title
# !datalad create ~/.tmp/$repository_title
!cp -r ~/WORKFLOW/PACKAGE/. ~/.tmp/$repository_title/

%cd ~/.tmp/$repository_title
!git add ~/.tmp/$repository_title
!git commit -m '1st commit'
!git remote add $repository_title $ssh_url
!git push $repository_title master
# !datalad save --to-git -m "1st commit"
# !datalad push

## 3. 上記リポジトリをサブデータセットとして設定する

In [ ]:
%cd ~/
!git submodule add $ssh_url ./experiments/$repository_title

## 4. pipeline.jsonに実験名を追記する

In [ ]:
import json

with open('/home/jovyan/experiments/pipeline.json', 'r') as f:
    pipeline = json.load(f)

pipeline.append(repository_title)

with open('/home/jovyan/experiments/pipeline.json', 'w') as f:
    json.dump(pipeline, f, indent = 4)

## 5. 変更内容を書き戻す

In [ ]:
%cd ~/
!git add ~/
!git commit -m '実験の作成'
!git push origin master
# !datalad save --to-git -m "実験の作成"
# !datalad push

## 6. 複製したパッケージのコンテナを立ち上げて、ソースコードのJupyterNotebookに遷移する

以下のセルを実行して表示されるリンクを押下してください。

In [ ]:
import urllib

print(
    "https://binder.cs.rcos.nii.ac.jp/v2/git/" + urllib.parse.quote(ssh_url, safe='') + "/HEAD"
)